## We first create a normal table and then move towards creating the 3nf table

In [97]:
import psycopg2

In [98]:
try:
    conn=psycopg2.connect('host=127.0.0.1 dbname=data_engineering user=postgres password=macbook')
except psycopg2.Error as e:
    print(e)

In [99]:
cur=conn.cursor()

In [100]:
conn.set_session(autocommit=True)

#### Let's imagine we have a table called Music Library. 

`Table Name: music_library
column 0: Album Id
column 1: Album Name
column 2: Artist Name
column 3: Year 
column 4: List of songs`


In [45]:
try:
    cur.execute("create table if not exists music_library(album_id int,album_name varchar,artist_name varchar,year int,song text[])")
except psycopg2.Error as e:
    print(e)

In [46]:
query="insert into music_library(album_id,album_name,artist_name,year,song)values(%s,%s,%s,%s,%s),"

### Insert values into the tables

In [47]:
query="INSERT INTO music_library (album_id, album_name, artist_name, year, song) VALUES (%s, %s, %s, %s, %s)"


In [48]:
try: 
    cur.execute(query,
                 (1, "Rubber Soul", "The Beatles", 1965, ["Michelle", "Think For Yourself", "In My Life"]))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [49]:
try: 
    cur.execute(query,
                 (2, "Let It Be", "The Beatles", 1970, ["Let It Be", "Across The Universe"]))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

#### Conform data is present in db, Use select * in postgres not in cassandra(NoSql)

In [50]:
try:
    cur.execute("select * from music_library")
except psycopg2.Error as e:
    print(e)

In [51]:
content=cur.fetchall()
for i in content:
    print(i)

(1, 'Rubber Soul', 'The Beatles', 1965, ['Michelle', 'Think For Yourself', 'In My Life'])
(2, 'Let It Be', 'The Beatles', 1970, ['Let It Be', 'Across The Universe'])


#### Moving to 1st Normal Form (1NF)
This data has not been normalized. To get this data into 1st normal form, we will need to remove any collections or list of data. We need to break up the list of songs into individuals rows. 


`Table Name: music_library2
column 0: Album Id
column 1: Album Name
column 2: Artist Name
column 3: Year 
column 4: Song Name`


In [52]:
try:
    cur.execute("create table if not exists music_library2(album_id int,album_name varchar,artist_name varchar,year int, song_name varchar)")
except psycopg2.Error as e:
    
    print(e)

In [54]:
query="insert into music_library2(album_id,album_name,artist_name,year,song_name)values(%s,%s,%s,%s,%s)"

In [55]:
try:
    cur.execute(query,(1, "Rubber Soul", "The Beatles", 1965, "Michelle"))
except psycopg2.Error as e:
    print(e)


In [56]:
try:
    cur.execute(query,                 (1, "Rubber Soul", "The Beatles", 1965, "Think For Yourself"))
except psycopg2.Error as e:
    print(e)


In [57]:
try:
    cur.execute(query,                 
                 (1, "Rubber Soul", "The Beatles", 1965, "In My Life"))
except psycopg2.Error as e:
    print(e)


In [58]:
try:
    cur.execute(query,                 
                 (2, "Let It Be", "The Beatles", 1970, "Let It Be"))
except psycopg2.Error as e:
    print(e)


In [59]:
try:
    cur.execute(query,                 
                 (2, "Let It Be", "The Beatles", 1970, "Across The Universe"))
except psycopg2.Error as e:
    print(e)


In [60]:
try:
    cur.execute("select * from music_library2")
except psycopg2.Error as e:
    print(e)
values=cur.fetchall()
for i in values:
    print(i)

(1, 'Rubber Soul', 'The Beatles', 1965, 'Michelle')
(1, 'Rubber Soul', 'The Beatles', 1965, 'Think For Yourself')
(1, 'Rubber Soul', 'The Beatles', 1965, 'In My Life')
(2, 'Let It Be', 'The Beatles', 1970, 'Let It Be')
(2, 'Let It Be', 'The Beatles', 1970, 'Across The Universe')


#### Moving to 2nd Normal Form (2NF)
We have moved our data to be in 1NF which is the first step in moving to 2nd Normal Form. Our table is not yet in 2nd Normal Form. While each of our records in our table is unique, our Primary key (*album id*) is not unique. We need to break this up into two tables, *album library* and *song library*. 

`Table Name: album_library 
column 0: Album Id
column 1: Album Name
column 2: Artist Name
column 3: Year `

`Table Name: song_library
column 0: Song Id
column 1: Song Name
column 3: Album Id` 


In [61]:
try:
    cur.execute("create table if not exists album_library(album_id int,album_name varchar,artist_name varchar,year int)")
except psycopg2.Error as e:
    print(e)
    

In [62]:
try:
    cur.execute("create table if not exists song_library(song_id int,album_id int,song_name varchar)")
except psycopg2.Error as e:
    print(e)

In [63]:
query="INSERT INTO album_library (album_id, album_name, artist_name, year) \
                 VALUES (%s, %s, %s, %s)"

In [64]:
cur.execute(query,(1,"Rubber Soul","The Beatles",1935))

In [65]:
cur.execute(query,                 (2, "Let It Be", "The Beatles", 1970))

In [66]:
query="INSERT INTO song_library (song_id, album_id, song_name) \
                 VALUES (%s, %s, %s)"

In [67]:
cur.execute(query, (1, 1, "Michelle"))

In [69]:
cur.execute(                 query,(2, 1, "Think For Yourself"))

In [70]:
cur.execute(query,                 (3, 1, "In My Life"))

In [71]:
cur.execute(query,                 (4, 2, "Let It Be"))


In [72]:
cur.execute(query,                 (5, 2, "Across the Universe"))


In [73]:
cur.execute("select * from album_library")

In [76]:
row=cur.fetchall()
for i in row:
    print(i)

(1, 'Rubber Soul', 'The Beatles', 1935)
(2, 'Let It Be', 'The Beatles', 1970)


In [101]:
cur.execute("SELECT * FROM song_library;")
row=cur.fetchall()
for i in row:
    print(i)

ProgrammingError: relation "song_library" does not exist
LINE 1: SELECT * FROM song_library;
                      ^


In [78]:
cur.execute("select * from album_library join song_library on album_library.album_id=song_library.album_id")

In [79]:
row=cur.fetchall()
for i in row:
    print(i)

(1, 'Rubber Soul', 'The Beatles', 1935, 1, 1, 'Michelle')
(1, 'Rubber Soul', 'The Beatles', 1935, 2, 1, 'Think For Yourself')
(1, 'Rubber Soul', 'The Beatles', 1935, 3, 1, 'In My Life')
(2, 'Let It Be', 'The Beatles', 1970, 4, 2, 'Let It Be')
(2, 'Let It Be', 'The Beatles', 1970, 5, 2, 'Across the Universe')


#### Moving to 3rd Normal Form (3NF)
Check our table for any transitive dependencies. *Album_library* can move *Artist_name* to its own table, called *Artists*, which will leave us with 3 tables. 

`Table Name: album_library2 
column 0: Album Id
column 1: Album Name
column 2: Artist Id
column 3: Year `

`Table Name: song_library
column 0: Song Id
column 1: Song Name
column 3: Album Id`

`Table Name: artist_library
column 0: Artist Id
column 1: Artist Name `

In [80]:
cur.execute("create table if not exists album_library2(album_id int,album_name varchar,artist_id int,year int)")

In [81]:
cur.execute("create table if not exists artist_library(artist_id int,artist_name varchar)")

In [82]:
query="INSERT INTO album_library2 (album_id, album_name, artist_id, year) \
                 VALUES (%s, %s, %s, %s)"

In [84]:
cur.execute(query,                 (1, "Rubber Soul", 1, 1965))


In [85]:
cur.execute(query,                 (2, "Let It Be", 1, 1970))


In [86]:
query="INSERT INTO artist_library (artist_id, artist_name) \
                 VALUES (%s, %s)"

In [87]:
cur.execute(query,                 (1, "The Beatles"))


In [88]:
cur.execute("SELECT * FROM album_library2;")
row=cur.fetchall()
for i in row:
    print(i)

(1, 'Rubber Soul', 1, 1965)
(2, 'Let It Be', 1, 1970)


In [90]:
cur.execute("select * from song_library")
row=cur.fetchall()
for i in row:
    print(i)

(1, 1, 'Michelle')
(2, 1, 'Think For Yourself')
(3, 1, 'In My Life')
(4, 2, 'Let It Be')
(5, 2, 'Across the Universe')


In [96]:
cur.execute("select * from artist_library")
row=cur.fetchall()
for i in row:
    print(i)

InterfaceError: cursor already closed

In [92]:
cur.execute("select * from (artist_library join album_library2 on artist_library.artist_id=album_library2.artist_id)join song_library on album_library2.album_id=song_library.album_id")

In [93]:
row=cur.fetchall()
for i in row:
    print(i)

(1, 'The Beatles', 1, 'Rubber Soul', 1, 1965, 1, 1, 'Michelle')
(1, 'The Beatles', 1, 'Rubber Soul', 1, 1965, 2, 1, 'Think For Yourself')
(1, 'The Beatles', 1, 'Rubber Soul', 1, 1965, 3, 1, 'In My Life')
(1, 'The Beatles', 2, 'Let It Be', 1, 1970, 4, 2, 'Let It Be')
(1, 'The Beatles', 2, 'Let It Be', 1, 1970, 5, 2, 'Across the Universe')


In [94]:
try: 
    cur.execute("DROP table music_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table music_library2")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table album_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table song_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table album_library2")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table artist_library")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

In [95]:
cur.close()
conn.close()